In [90]:
import numpy as np
import pandas as pd
from random import shuffle
import praw
import reddit_tools
from psaw import PushshiftAPI
import datetime as dt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [2]:
reddit = praw.Reddit(client_id='',
                      client_secret='',
                      user_agent='test')

In [7]:
submissions = reddit_tools.grab_data('MachineLearning', reddit)
# submissions = np.load('MachineLearning_submissions.npy')

grabbed 37615 submissions


In [11]:
all_comments = reddit_tools.grab_comments(submissions)

0 out of 37615
2000 out of 37615
4000 out of 37615
6000 out of 37615
8000 out of 37615
10000 out of 37615
12000 out of 37615
14000 out of 37615
16000 out of 37615
18000 out of 37615
20000 out of 37615
22000 out of 37615
24000 out of 37615
26000 out of 37615
28000 out of 37615
30000 out of 37615
32000 out of 37615
34000 out of 37615
36000 out of 37615


In [81]:
shuffle(all_comments)

stop_words = np.load('stop_words.npy')
add_stop = ['good','work','time','machine','learning','thing','lot','problem','bad','doesnt','model']
lt = reddit_tools.LemmaTokenizer()
new_stop = list(stop_words)+add_stop
new_stop = lt(' '.join(new_stop))+add_stop

tf_vect = CountVectorizer(tokenizer=reddit_tools.LemmaTokenizer(),
                                strip_accents='unicode', 
                                stop_words=set(new_stop), 
                                lowercase=True,
                                min_df=20,max_df=.75,) 

search_params = {'lda__n_components': [7,10],
                'lda__learning_method':['online'], 'lda__learning_offset':[25]
                }

# Init the Model
lda = LatentDirichletAllocation()

lda_pipe = Pipeline([
    ('vect', tf_vect),
    ('lda', lda)
])


# Init Grid Search Class
model = GridSearchCV(lda_pipe, param_grid=search_params, verbose=10, n_jobs=-1)

# Do the Grid Search
model.fit(all_comments)



Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed: 27.9min remaining: 55.8min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 28.1min remaining: 28.1min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed: 28.2min remaining: 14.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 41.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 41.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.75, max_features=None, min_df=20,
        ngram_range=(1, 1), preprocessor=None,
        stop_words={'with', ...        random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lda__n_components': [7, 10], 'lda__learning_method': ['online'], 'lda__learning_offset': [25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [82]:
lda = model.best_estimator_
reddit_tools.display_topics(lda.named_steps['lda'], lda.named_steps['vect'].get_feature_names(), 15)


Topic 0:
game human agent action bot player reward policy algorithm question state environment reinforcement strategy play
Topic 1:
code tensorflow pytorch python project library implementation data easy kera framework tool graph version file
Topic 2:
gpu data gpus card google hardware price nvidia cpu memory cloud car deep performance cost
Topic 3:
people year company paper data job field researcher point phd idea conference google big community
Topic 4:
paper deep data language book idea video neural text math great project post algorithm question
Topic 5:
paper network neural function gradient layer method weight point deep parameter training author architecture number
Topic 6:
data image feature network training output dataset input sample distribution class loss classification case layer


In [86]:
res_df = topic_df(lda.named_steps['lda'], lda.named_steps['vect'].get_feature_names())

In [94]:
res_df.columns = ['Reinforcement Learning','Frameworks','Hardware','Career','??','Architecture/impementation','Feature Engineering']

In [95]:
res_df.head(10)

,Reinforcement Learning,Frameworks,Hardware,Career,??,Architecture/impementation,Feature Engineering
0,game,code,gpu,people,paper,paper,data
1,human,tensorflow,data,year,deep,network,image
2,agent,pytorch,gpus,company,data,neural,feature
3,action,python,card,paper,language,function,network
4,bot,project,google,data,book,gradient,training
5,player,library,hardware,job,idea,layer,output
6,reward,implementation,price,field,video,method,dataset
7,policy,data,nvidia,researcher,neural,weight,input
8,algorithm,easy,cpu,point,text,point,sample
9,question,kera,memory,phd,math,deep,distribution


In [93]:
pyLDAvis.sklearn.prepare(
    lda.named_steps['lda'], 
    lda.named_steps['vect'].fit_transform(all_comments), 
    lda.named_steps['vect'])

/Users/vish/anaconda/envs/datascience/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.074537 -0.159289       1        1  22.987483
5      0.150674  0.042350       2        1  19.309456
6      0.162232  0.120239       3        1  17.947456
4      0.025902 -0.087583       4        1  15.545105
1     -0.124841  0.060517       5        1   9.679712
0      0.060373 -0.089125       6        1   8.523846
2     -0.199804  0.112892       7        1   6.006942, topic_info=     Category          Freq            Term         Total  loglift  logprob
2783  Default  11319.000000           image  11319.000000  30.0000  30.0000
1047  Default   8552.000000            code   8552.000000  29.0000  29.0000
2381  Default   5922.000000            game   5922.000000  28.0000  28.0000
4083  Default  23309.000000           paper  23309.000000  27.0000  27.0000
2719  Default   6725.000000           human   6725.000000  26.0000  26.0000
1464  Default  21239.000000            data  21239.000000  25.0000  25.0000
3776  Default  16153.000000         network  16153.000000  24.0000  24.0000
4160  Default  15732.000000          people  15732.000000  23.0000  23.0000
5659  Default   3612.000000      tensorflow   3612.000000  22.0000  22.0000
278   Default   2865.000000           agent   2865.000000  21.0000  21.0000
4551  Default   2973.000000         pytorch   2973.000000  20.0000  20.0000
2498  Default   4858.000000        gradient   4858.000000  19.0000  19.0000
4549  Default   2854.000000          python   2854.000000  18.0000  18.0000
2493  Default   2685.000000             gpu   2685.000000  17.0000  17.0000
212   Default   2763.000000          action   2763.000000  16.0000  16.0000
2364  Default   7595.000000        function   7595.000000  15.0000  15.0000
4471  Default   6355.000000         project   6355.000000  14.0000  14.0000
755   Default   3922.000000             bot   3922.000000  13.0000  13.0000
3226  Default   6594.000000           layer   6594.000000  12.0000  12.0000
3778  Default   9254.000000          neural   9254.000000  11.0000  11.0000
3283  Default   2427.000000         library   2427.000000  10.0000  10.0000
1515  Default   9048.000000            deep   9048.000000   9.0000   9.0000
2177  Default   5569.000000         feature   5569.000000   8.0000   8.0000
1117  Default   4215.000000         company   4215.000000   7.0000   7.0000
3204  Default   3853.000000        language   3853.000000   6.0000   6.0000
4267  Default   1844.000000          player   1844.000000   5.0000   5.0000
4854  Default   1785.000000          reward   1785.000000   4.0000   4.0000
2482  Default   4317.000000          google   4317.000000   3.0000   3.0000
738   Default   2593.000000            book   2593.000000   2.0000   2.0000
2809  Default   3447.000000  implementation   3447.000000   1.0000   1.0000
...       ...           ...             ...           ...      ...      ...
3969   Topic7    219.476492          opencl    232.988819   2.7525  -6.3010
5795   Topic7    292.873787             tpu    320.062351   2.7235  -6.0125
2591   Topic7   1066.594344        hardware   1300.305480   2.6141  -4.7200
4605   Topic7    380.680000             ram    430.377471   2.6896  -5.7503
1789   Topic7    398.460030          driver    492.360689   2.6006  -5.7047
4441   Topic7    228.749190       processor    258.207129   2.6911  -6.2596
5097   Topic7    630.189717          server    932.787179   2.4201  -5.2463
1616   Topic7    490.687511          device    697.253948   2.4609  -5.4965
3286   Topic7    315.689421         license    399.270689   2.5774  -5.9375
5674   Topic7    306.822247           tesla    384.456362   2.5867  -5.9660
941    Topic7    463.519056           cheap    734.896927   2.3514  -5.5534
5098   Topic7    577.604086         service   1036.698489   2.2273  -5.3334
873    Topic7    780.541551             car   1751.781346   2.0039  -5.0323
1320   Topic7    557.410249            core   1044.